In [ ]:
# http://statsmodels.sourceforge.net/stable/datasets/generated/fair.html

In [ ]:
import statsmodels.api as sm
from sklearn import linear_model
import numpy as np
import pandas as pd
from patsy import dmatrices
from sklearn import metrics
%pylab inline

In [ ]:
import sys
import matplotlib as mpl
print("python " + sys.version)
print("")
print("pandas " + str(pd.__version__))
print("numpy " + np.__version__)

In [ ]:
df = sm.datasets.fair.load_pandas().data

In [ ]:
df.head(4)

##### data exploration

In [ ]:
# some summary statistics.
#misleading - [ explained towards the end. bottom line - exercise caution while interpreting summary stats ]

In [ ]:
df.describe()

In [ ]:
#need to run this to see the figures. 
#%matplotlib inline

import matplotlib.pyplot as plt

axes = pd.tools.plotting.scatter_matrix(df, alpha=0.2)
plt.tight_layout()
plt.savefig('scatter_matrix.png')
#plt.show()

In [ ]:
df['affair_bool'] = (df.affairs > 0).astype(int)

In [ ]:
df['affairs'].hist(bins=20)
xlabel('affairs')
ylabel('number of women')

In [ ]:
#Exploring affairs vs marraige ratings : 

In [ ]:
rating_vs_target = pd.crosstab(df['rate_marriage'], df['affair_bool'])
rating_vs_target

In [ ]:
total_ratings = rating_vs_target.apply(sum)
total_ratings

In [ ]:
rating_vs_target /= total_ratings
rating_vs_target

In [ ]:
rating_vs_target.plot(kind='bar')

##### Conclusion - Women who rate their marriages higher have lesser number of affairs. Let use a model to verify our claim.

In [ ]:
#splitting the model into train and test sets

In [ ]:
#set the random state to get the same split each time
from sklearn.cross_validation import train_test_split
x_train, x_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_y = x_train["affair_bool"]

In [ ]:
test_y = x_test["affair_bool"]

In [ ]:
x_train = x_train.drop("affairs", axis = 1, inplace = False)
x_train = x_train.drop("affair_bool", axis = 1, inplace = False)

In [ ]:
x_test = x_test.drop("affairs", axis = 1, inplace = False)
x_test = x_test.drop("affair_bool", axis = 1, inplace = False)

In [ ]:
#fitting the model

In [ ]:
logit = linear_model.LogisticRegression()
logit.fit(x_train,train_y)

In [ ]:
#making predictions. By default, 0.5 is chosen as the threshold

In [ ]:
predicted = logit.predict(x_test)

In [ ]:
#calculating accuracy

In [ ]:
np.mean((predicted - test_y)**2)

In [ ]:
#coefficients

In [ ]:
logit.coef_

In [ ]:
logit.intercept_

In [ ]:
weights = pd.Series(logit.coef_[0],
                 index= x_train.columns.values)
weights.sort_values()

##### Conclusions - We observed from the bar graph that a higher marriage rating means a lower chance of having an affair. The negative coefficient value of rate_marriage variable - -0.683867 tells us exactly that. But are we on the right track?

In [ ]:
# We observe that the occupation husb and occupation columns are categorical columns. We need to make them categorical so we will use design matrices.
# The C(occupation_husb) and C(occupation) is doing just that

In [ ]:
y, X = dmatrices('affair_bool ~ rate_marriage + age + educ + children + C(occupation_husb) + C(occupation) + yrs_married', df, return_type = 'dataframe')

In [ ]:
logit_categorical = linear_model.LogisticRegression(fit_intercept = False, C = 1e9)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
logit_categorical.fit(X_train,y_train)

In [ ]:
logit_categorical.coef_

In [ ]:
weights_categories = pd.Series(logit_categorical.coef_[0],
                 index= X.columns.values)
weights_categories.sort_values()

In [ ]:
# This makes more sense. Let's calculate the accuracy insample 

In [ ]:
logit_categorical.score(X_train, y_train)

In [ ]:
# Let's now check the out of sample test accuracy 

In [ ]:
predicted_logit_categorical = logit_categorical.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, predicted_logit_categorical)

In [ ]:
#getting the predicted probabilities.
predicted_prob = logit_categorical.predict_proba(X_test)

##### How do we choose the cut off threshold? 
##### 1. Depends on the business problem
##### 2. Related to 1 - How much error are we willing to make by predicting either class


In [ ]:
predicted_class_1 = [item[1] for item in predicted_prob]


In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, predicted_class_1)
roc_auc = metrics.auc(fpr,tpr)

In [ ]:
#adapted from http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('roc.png')
plt.show()


### REGULARIZATION 

### Switch to Emma

What if the effect of husband's occupation depends on the wife's occupation, or the effect of children depends on years married? 

We can add features to include interaction terms. First let's have a look at what feature we have now.

In [ ]:
X_train.head()

We do not want to create interaction terms with intercept because that will simply be the feature itself. Lets drop be intercept column first.

In [ ]:
X_train= X_train.drop('Intercept', axis=1)
X_test= X_test.drop('Intercept', axis=1)

In [ ]:
print X_train.shape 
print X_test.shape 

We can see the range of values is different for each numerical variables. Since the values of coefficience matter in regularization, we want to standardize the data first (only numerical variables). 

In [ ]:
for i in ['rate_marriage','age','educ','children','yrs_married']:
    X_train[i]= (X_train[i] - X_train[i].mean()) / X_train[i].std()
    X_test[i]= (X_test[i] - X_test[i].mean()) / X_test[i].std()

Ideally we want to standardize training/testing data the same way. But considering after training the model one may not keep the original training data, we standardized the testing data with its own mean/std.


In [ ]:
X_train.head()

We can see the current dataset has 15 features. With PolynomialFeatures, we can create all second order features in one command. 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
X_train_2 = poly.fit_transform(X_train)
X_test_2 = poly.fit_transform(X_test)

In [ ]:
print X_train_2.shape 
print X_test_2.shape 

There is 1 feature for the intercept, 15 original features, 15 second order features for the original 15 features, and 15-choose-2(105) second order interacting features. The total number of features is 136 as expected.

The outputs of PolynomialFeatures are numpy arrays. Let's add the column names back and make some dataframes. 

In [ ]:
target_feature_names = poly.get_feature_names(X_train.columns.values)
X_train_2_df = pd.DataFrame(X_train_2, columns = target_feature_names)
X_test_2_df = pd.DataFrame(X_test_2, columns = target_feature_names)

In [ ]:
X_train_2_df.head()

Now let's fit a new logistic regression with our freshly made features :) For easy comparison, we use the same parameters as used in the previous model.

In [ ]:
model = linear_model.LogisticRegression(fit_intercept = False, penalty = 'l2', C= 1e9)
model.fit(X_train_2_df,y_train)

print 'train_score = ', model.score(X_train_2_df, y_train)
    
predicted = model.predict(X_test_2_df)
print 'test_score = ', metrics.accuracy_score(y_test, predicted)

What are the most important features this time?

In [ ]:
weights_categories = pd.Series(model.coef_[0],
                 index= X_train_2_df.columns.values)
topindices = np.argsort(-np.abs(model.coef_[0]))[:10]
weights_categories[topindices][:10]

The testing error is a bit larger than the training error. We may be overfitting the dataset. Why don't we try some regularization? 

In [ ]:
def fit_order2(pen,reg):
    model = linear_model.LogisticRegression(fit_intercept = False, penalty = pen, C= reg)
    model.fit(X_train_2_df,y_train)
    
    #print model.coef_
    train_score = model.score(X_train_2_df, y_train)
    
    predicted = model.predict(X_test_2_df)
    test_score = metrics.accuracy_score(y_test, predicted)
    
    return train_score, test_score, model.coef_[0]

Let's train a suite of models with various regularization strength.

In [ ]:
penalties = [1e9, 1e6, 1e4, 1e2, 10, 1, 1e-1, 1e-2, 1e-3, 1e-4]
types = ['l1', 'l2']

train_score = np.zeros([len(types),len(penalties)])
test_score = np.zeros([len(types),len(penalties)])
coeff = np.zeros([len(types),len(penalties),X_train_2_df.shape[1]])

In [ ]:
for i in range(len(types)):
    for j in range(len(penalties)):
        (train_score[i,j], test_score[i,j], coeff[i,j,:]) = fit_order2(types[i], penalties[j])

In [ ]:
print train_score[0,:]
print test_score[0,:]

Let's see how the accuracy and coefficents change with the penalty. We want to plot the lambda instead of C for this purpose.

In [ ]:
actual_pen = [1/i for i in penalties]

In [ ]:
actual_pen

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 12, 8

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot()
ydata =[train_score[0,:], test_score[0,:], train_score[1,:], test_score[1,:]]
colors =['red', 'black', 'red','black']
markers = ['o','o','^','^']
labels =[ 'L1 train','L1 test','L2 train','L2 test']
for i in range(4):
    ax.scatter(actual_pen,ydata[i], color= colors[i] ,label = labels[i],marker=markers[i], s=50)
    ax.plot(actual_pen, ydata[i], color=colors[i])


ax.set_xscale('log')
plt.xlim([1e-9, 1e5])
plt.xlabel('Lambda (penalty)', fontsize=25)
plt.ylabel('Accuracy', fontsize=25)
plt.legend(loc=3) #, fontsize=25
plt.rcParams.update({'font.size': 25})
plt.show()

In [ ]:
# let's extract top 10 most important word for l2, week reguarlization
l1indices = np.argsort(-np.abs(coeff[0,0,:]))[:10]
l2indices = np.argsort(-np.abs(coeff[1,0,:]))[:10]
#print indices
l1coeff = coeff[0,:,l1indices]
l2coeff = coeff[1,:,l2indices]

In [ ]:
l1words = weights_categories[l1indices].index
l2words = weights_categories[l2indices].index
print 'L1 regularization'
print l1words
print ''
print 'L2 regularization'
print l2words

In [ ]:
plt.rcParams['figure.figsize'] = 16, 10

def make_coefficient_plot(coeff, words, penalty_list, title):
    
    cmap = plt.get_cmap('Blues')
    
    xx = penalty_list
    plt.plot(xx, [0.]*len(xx), '--', lw=1, color='k')
    
    table_words = coeff

    for i in xrange(len(words)):
        color = cmap(0.8*((i+1)/(len(words)*1.2)+0.15))
        plt.plot(xx, coeff[i:i+1].flatten(),
                 '-', label=words[i], linewidth=4.0, color=color)
        
    plt.legend(loc=1, ncol=2, prop={'size':16}, columnspacing=0.5)
    plt.axis([1, 1e4, -1, 2])
    #plt.ylim([-0.5,0.5])
    plt.xlim([1, 1e4])
    plt.title(title)
    plt.xlabel('Lambda (penalty)')
    plt.ylabel('Coefficient value')
    plt.xscale('log')

    plt.rcParams.update({'font.size': 25})
    plt.tight_layout()

In [ ]:
make_coefficient_plot(l1coeff, l1words, actual_pen, 'L1 penalty')

In [ ]:
make_coefficient_plot(l2coeff, l2words, actual_pen, 'L2 penalty')

### Caveats:
There are only ~6k data points in this dataset, so the difference in training/testing error could just be noise.

We will do better by doing cross-validation, but it's hard to gain much just because we don't have much data to start with.